<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [ ]:
# change these to try this notebook out
BUCKET = 'asl-ml-immersion-temp'
PROJECT = 'asl-ml-immersion'
REGION = 'us-central1'

In [ ]:
# Import os environment variables
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [ ]:
import google.datalab.bigquery as bq
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  mother_race,
  plurality,
  gestation_weeks,
  mother_married,
  ever_born,
  cigarette_use,
  alcohol_use,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. How many unique values of hashmonth do you have?

In [ ]:
# TODO: Find the number of unique values per hashmonth


So, getting 10,000 samples is not as simple as just doing MOD(hashmonth, 10000) == 1.
<p>
Get a well distributed portion (approximately 10,000) of the data in such a way that the test and train sets do not overlap.

In [ ]:
# TODO: Create train and eval queries to create well distributed portions with approximately 10,000 rows of the data without overlap


<h2> Preprocess data using Pandas </h2>
<p>
Do the following preprocessing of the dataframes:
<ol>
<li>
Notice that the race field is 1.0, 4.0, etc.  Let's replace by text-strings such as "White" or "Asian Indian". Our final deployed service will ask for text names, not magic numbers.
<li>
Remove any unwanted columns, i.e. columns without predictive power.  Remove the ever_born column.
<li>
Make sure to not use rows that are missing any of the following very key fields:  weight_pounds, mother_age, gestation_weeks, plurality
</ol>

In [ ]:
# TODO: Invesigate data and figure out what to remove/filter/replace/etc.


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [ ]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [ ]:
%bash
wc -l *.csv
head *.csv

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License